In [ ]:
"""
Retrieve and output geocoordinates of various MRT stations and shopping malls in Singapore
for subsequent use
"""

In [1]:
# Import libraries
import math
import json
import requests
import pandas as pd
import numpy as np
import os

In [2]:
# Set display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

In [3]:
# Change directories
os.chdir(r'/Users/sawleen/Documents/Leen/MTech EBAC NUS ISS/Sem 1 2021/Practice Module/Data')

In [4]:
############ Get coordinates of MRT stations ############

In [5]:
# Initialize empty dataframe to store details
mrt_location=pd.DataFrame([],columns=['MRT','latitude','longitude'])

# Get list of MRT to search for
mrt_data=pd.read_csv('train-station-chinese-names.csv') #Downloaded from data.gov.sg
mrt_data.head()

,stn_code,mrt_station_english,mrt_station_chinese,mrt_line_english,mrt_line_chinese
0,NS1,Jurong East,裕廊东,North South Line,南北线
1,NS2,Bukit Batok,武吉巴督,North South Line,南北线
2,NS3,Bukit Gombak,武吉甘柏,North South Line,南北线
3,NS4,Choa Chu Kang,蔡厝港,North South Line,南北线
4,NS5,Yew Tee,油池,North South Line,南北线


In [6]:
# Eyeball to see if data is complete
print('Number of MRT stations downloaded: {}'.format(len(mrt_data)))
print(mrt_data.mrt_station_english.unique())

Number of MRT stations downloaded: 184
['Jurong East' 'Bukit Batok' 'Bukit Gombak' 'Choa Chu Kang' 'Yew Tee'
 'Kranji' 'Marsiling' 'Woodlands' 'Admiralty' 'Sembawang' 'Yishun'
 'Khatib' 'Yio Chu Kang' 'Ang Mo Kio' 'Bishan' 'Braddell' 'Toa Payoh'
 'Novena' 'Newton' 'Orchard' 'Somerset' 'Dhoby Ghaut' 'City Hall'
 'Raffles Place' 'Marina Bay' 'Marina South Pier' 'Pasir Ris' 'Tampines'
 'Simei' 'Tanah Merah' 'Bedok' 'Kembangan' 'Eunos' 'Paya Lebar' 'Aljunied'
 'Kallang' 'Lavender' 'Bugis' 'Tanjong Pagar' 'Outram Park' 'Tiong Bahru'
 'Redhill' 'Queenstown' 'Commonwealth' 'Buona Vista' 'Dover' 'Clementi'
 'Chinese Garden' 'Lakeside' 'Boon Lay' 'Pioneer' 'Joo Koon' 'Gul Circle'
 'Tuas Crescent' 'Tuas West Road' 'Tuas Link' 'Expo' 'Changi Airport'
 'HarbourFront' 'Chinatown' 'Clarke Quay' 'Little India' 'Farrer Park'
 'Boon Keng' 'Potong Pasir' 'Woodleigh' 'Serangoon' 'Kovan' 'Hougang'
 'Buangkok' 'Sengkang' 'Punggol' 'Bras Basah' 'Esplanade' 'Promenade'
 'Nicoll Highway' 'Stadium' 'Mountbatte

In [7]:
# Drop unnecessary columns
mrt_data.drop(['mrt_station_chinese','mrt_line_chinese'],axis=1,inplace=True)

# Clean MRT station column to prepare for search value
mrt_data['stn_name_clean']=''
for i in range(len(mrt_data)):
    mrt_stn=mrt_data.iat[i,1] #mrt_station_english
    mrt_line=mrt_data.iat[i,2] #mrt_line_english
    
    # Append with suffix 'LRT' or 'MRT' station
    if mrt_line.endswith('LRT'):
        mrt_data.iat[i,3]=mrt_stn + ' LRT Station'
    else:
        mrt_data.iat[i,3]=mrt_stn + ' MRT Station'

In [8]:
# Count missing values that require cleaning
print(pd.isna(mrt_data)['mrt_station_english'].sum()) #0

"""
# Clean nan values (but not necessary as no missing values)
mrt_stations_clean=mrt_data.copy()
mrt_stations_clean['stn_name_clean'].replace('',np.nan,inplace=True)
mrt_stations_clean.dropna(subset=['stn_name_clean'],inplace=True)
print('Number of MRT stations cleaned: {}'.format(len(mrt_stations_clean)))
"""

0


"\n# Clean nan values (but not necessary as no missing values)\nmrt_stations_clean=mrt_data.copy()\nmrt_stations_clean['stn_name_clean'].replace('',np.nan,inplace=True)\nmrt_stations_clean.dropna(subset=['stn_name_clean'],inplace=True)\nprint('Number of MRT stations cleaned: {}'.format(len(mrt_stations_clean)))\n"

In [9]:
# Get cleaned list of MRT station names
list_of_mrt=mrt_data.stn_name_clean.unique()
list_of_mrt

array(['Jurong East MRT Station', 'Bukit Batok MRT Station',
       'Bukit Gombak MRT Station', 'Choa Chu Kang MRT Station',
       'Yew Tee MRT Station', 'Kranji MRT Station',
       'Marsiling MRT Station', 'Woodlands MRT Station',
       'Admiralty MRT Station', 'Sembawang MRT Station',
       'Yishun MRT Station', 'Khatib MRT Station',
       'Yio Chu Kang MRT Station', 'Ang Mo Kio MRT Station',
       'Bishan MRT Station', 'Braddell MRT Station',
       'Toa Payoh MRT Station', 'Novena MRT Station',
       'Newton MRT Station', 'Orchard MRT Station',
       'Somerset MRT Station', 'Dhoby Ghaut MRT Station',
       'City Hall MRT Station', 'Raffles Place MRT Station',
       'Marina Bay MRT Station', 'Marina South Pier MRT Station',
       'Pasir Ris MRT Station', 'Tampines MRT Station',
       'Simei MRT Station', 'Tanah Merah MRT Station',
       'Bedok MRT Station', 'Kembangan MRT Station', 'Eunos MRT Station',
       'Paya Lebar MRT Station', 'Aljunied MRT Station',
       'Kal

In [10]:
# Find coordinates of MRT stations
for i in range(0, len(list_of_mrt)):
    # Initialize
    query_address = list_of_mrt[i]
    if query_address=='': #Skip if blank
        continue

    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)
    data_mrt=json.loads(resp.content)

    mrt_lat=None
    mrt_long=None

    if data_mrt['found'] != 0:
        mrt_lat=data_mrt["results"][0]["LATITUDE"]
        mrt_long=data_mrt["results"][0]["LONGITUDE"]

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        print ("No Results")

    # Store information in a dataframe
    mrt_location_current = {
        'MRT': list_of_mrt[i],
        'latitude': mrt_lat,
        'longitude': mrt_long}
    mrt_location_current_df=pd.DataFrame.from_dict(mrt_location_current,orient='index').T
    # Append to main dataframe
    mrt_location=mrt_location.append(mrt_location_current_df)

Jurong East MRT Station,Lat: 1.33357652289465 Long: 103.742292350324
Bukit Batok MRT Station,Lat: 1.34842377665437 Long: 103.749126361849
Bukit Gombak MRT Station,Lat: 1.35930937732431 Long: 103.751948590322
Choa Chu Kang MRT Station,Lat: 1.38575650064861 Long: 103.744509334035
Yew Tee MRT Station,Lat: 1.39753506936297 Long: 103.747405150236
Kranji MRT Station,Lat: 1.42513172976083 Long: 103.762492227566
Marsiling MRT Station,Lat: 1.43275722977074 Long: 103.773981720213
Woodlands MRT Station,Lat: 1.43659643136072 Long: 103.78584424633
Admiralty MRT Station,Lat: 1.44053338763298 Long: 103.801364534318
Sembawang MRT Station,Lat: 1.44892733136332 Long: 103.820401560553
Yishun MRT Station,Lat: 1.42966559741799 Long: 103.83564365509
Khatib MRT Station,Lat: 1.41789179005502 Long: 103.833171755256
Yio Chu Kang MRT Station,Lat: 1.3819602145119 Long: 103.844651533361
Ang Mo Kio MRT Station,Lat: 1.36988737282394 Long: 103.849825701795
Bishan MRT Station,Lat: 1.35057959895607 Long: 103.8483049918

Teck Whye LRT Station,Lat: 1.37668467920808 Long: 103.753712232337
Phoenix LRT Station,Lat: 1.37861545104083 Long: 103.757995558346
Bukit Panjang LRT Station,Lat: 1.37792694616982 Long: 103.763102931112
Petir LRT Station,Lat: 1.37777204259082 Long: 103.766645782497
Pending LRT Station,Lat: 1.37613574334988 Long: 103.771261175571
Bangkit LRT Station,Lat: 1.38002223010088 Long: 103.772647370452
Fajar LRT Station,Lat: 1.38457317668025 Long: 103.770887223018
Segar LRT Station,Lat: 1.38778508663362 Long: 103.769599685249
Jelapang LRT Station,Lat: 1.38673926997961 Long: 103.76453408646
Senja LRT Station,Lat: 1.38272547134453 Long: 103.76234425472
No Results
Sengkang LRT Station,Lat: 1.39160936379906 Long: 103.895442583336
Compassvale LRT Station,Lat: 1.39449304450373 Long: 103.900492450944
Rumbia LRT Station,Lat: 1.39146849534132 Long: 103.90597359341
Bakau LRT Station,Lat: 1.38799431054769 Long: 103.905415300171
Kangkar LRT Station,Lat: 1.38395911688915 Long: 103.90222537044
Ranggung LRT St

In [11]:
# Visualize output
mrt_location.reset_index(drop=True,inplace=True)
mrt_location.head()

,MRT,latitude,longitude
0,Jurong East MRT Station,1.33357652289465,103.742292350324
1,Bukit Batok MRT Station,1.34842377665437,103.749126361849
2,Bukit Gombak MRT Station,1.35930937732431,103.751948590322
3,Choa Chu Kang MRT Station,1.38575650064861,103.744509334035
4,Yew Tee MRT Station,1.39753506936297,103.747405150236


In [12]:
# Save to CSV for future use
mrt_location.to_csv('mrt_locations.csv',index=False)

In [13]:
############ Get coordinates of shopping malls ############

In [14]:
### Get shopping mall locations
# Shopping malls taken from Wikipedia, correct as of Aug 2020
list_of_malls = [
    '100 AM', '313@Somerset', 'Aperia', 'Balestier Hill Shopping Centre', 'Bugis Cube', 'Bugis Junction', 'Bugis+',
    'Capitol Piazza', 'Cathay Cineleisure Orchard', 'City Gate', 'City Square Mall', 'CityLink Mall', 'The Central',
    'Duo', 'Far East Plaza', 'Funan', 'Great World City', 'HDB Hub', 'Holland Village', 'ION Orchard', 'Junction 8',
    'Knightsbridge', 'Liang Court', 'Liat Towers', 'Lucky Plaza', 'Marina Bay Financial Centre Tower 3', 'Marina Bay Link Mall',
    'Marina Bay Sands', 'Marina One', 'Marina Square', 'Midpoint Orchard', 'Millenia Walk', 'Mustafa', 'Ngee Ann City',
    'Orchard Central', 'Orchard Gateway', 'Orchard Plaza', 'Orchard Shopping Centre', 'Palais Renaissance',
    "People's Park Centre", "People's Park Complex", 'Plaza Singapura', 'Pomo', 'Raffles City',
    'Scotts Square', 'Serangoon Plaza', 'Shaw House', 'Sim Lim Square', 'Singapore Shopping Centre', 'Square 2', 'Suntec City',
    'Tanglin Mall', 'Tangs', 'Tanjong Pagar Centre', 'Tekka Centre', 'The Centrepoint', 'The Paragon', 'The Poiz',
    'The Shoppes at Marina Bay Sands', 'The South Beach', 'Thomson Plaza', 'United Square', 'Velocity',
    'Wheelock Place', 'Wisma Atria', 'Zhongshan Mall',
    '112 Katong', 'Bedok Mall', 'Bedok Point', 'Century Square', 'Changi Airport', 'Changi City Point', 'City Plaza',
    'Djitsun Mall Bedok', 'Downtown East', 'East Village', 'Eastpoint Mall', 'Elias Mall', 'Kallang Wave Mall',
    'Katong Square', 'Katong V', 'KINEX', 'Leisure Park Kallang', 'Loyang Point', 'Our Tampines Hub',
    'Parkway Parade', 'Paya Lebar Square', 'PLQ Mall', 'Singapore Post Centre', 'Tampines 1', 'Tampines Mall',
    'The Flow', 'White Sands', '888 Plaza', 'Admiralty Place', 'AMK Hub', 'Beauty World Centre', 'Beauty World Plaza',
    'Broadway Plaza', 'Buangkok Square', 'Bukit Panjang Plaza', 'Bukit Timah Plaza', 'Causeway Point', 'Compass One',
    'Djitsun Mall', 'Fajar Shopping Centre', 'Greenridge Shopping Centre', 'Greenwich V', 'Heartland Mall', 'Hillion Mall',
    'HillV2', 'Hougang 1', 'Hougang Green Shopping Mall', 'Hougang Mall', 'Jubilee Square', 'Junction 10', 'Junction 9',
    'Keat Hong Shopping Centre', 'Limbang Shopping Centre', 'Lot One', 'Marsiling Mall', 'myVillage Serangoon',
    'NEX', 'Northpoint City', 'Oasis Terraces', 'Punggol Plaza', 'Rail Mall', 'Rivervale Mall',
    'Rivervale Plaza', 'Sembawang Shopping Centre', 'Sun Plaza', 'Sunshine Place', 'Teck Whye Shopping Centre', 'The Midtown',
    'The Seletar Mall', 'Upper Serangoon Shopping Centre', 'Waterway Point', 'West Mall', 'Wisteria Mall', 'Woodlands Mart',
    'Yew Tee Point', 'Yew Tee Square', 'Alexandra Retail Centre', 'HarbourFront Centre', 'VivoCity', '321 Clementi',
    'Alexandra Central', 'Anchorpoint', 'Big Box', 'Boon Lay Shopping Centre', 'Fairprice',
    'Gek Poh Shopping Centre', 'Grantral Mall', 'IMM', 'JCube', 'Jem', 'Jurong Point', 'Grandstand', 'Pioneer Mall',
    'Queensway Shopping Centre', 'Rochester Mall', 'Taman Jurong Shopping Centre', 'The Clementi Mall', 'The Star Vista',
    'Tiong Bahru Plaza', 'West Coast Plaza', 'Westgate']

In [15]:
# Function to fetch coordinates of malls
def get_mall_coord(list_of_shopping_mall):
    # Initialize
    mall_location=pd.DataFrame([],columns=['mall_name', 'searchval','latitude', 'longitude', 'postal_code','road_name'])
    malls_not_found=[]

    for i in range(0, len(list_of_shopping_mall)):
        # Initialize
        query_address = list_of_shopping_mall[i]
        query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
        resp = requests.get(query_string)
        data_mall=json.loads(resp.content)

        mall_location_current = {
            'mall_name': list_of_shopping_mall[i],
            'searchval':None,
            'latitude': None,
            'longitude': None,
            'postal_code':None,
            'road_name':None}

        # Find number of results found
        if 'found' in data_mall:
            results_num = data_mall['found']
        else:
            results_num = 0

        # Find details if there are results
        if results_num != 0:
            limit=min(10,results_num) # Set limit as 10

            for j in range(limit):
                mall_postal=data_mall['results'][j]['POSTAL'].strip()

                if (mall_postal.isdigit() and len(mall_postal)==6) or i==9:
                    searchval=data_mall["results"][j]["SEARCHVAL"]
                    mall_lat=data_mall["results"][j]["LATITUDE"]
                    mall_long=data_mall["results"][j]["LONGITUDE"]
                    mall_postal=data_mall["results"][j]["POSTAL"]
                    road_name=data_mall['results'][j]['ROAD_NAME']

                    print (str(query_address)+", searchval: "+searchval +" road_name: "+road_name)

                    # Store information in a dataframe
                    mall_location_current = {
                        'mall_name': list_of_shopping_mall[i],
                        'searchval':searchval,
                        'latitude': mall_lat,
                        'longitude': mall_long,
                        'postal_code':mall_postal,
                        'road_name':road_name}
                    break

                else:
                    continue

        else:
            print ("No Results")
            malls_not_found.append(query_address)


        mall_location_current_df=pd.DataFrame.from_dict(mall_location_current,orient='index').T
        # Append to main dataframe
        mall_location=mall_location.append(mall_location_current_df)

    return [mall_location, malls_not_found]

In [16]:
# Fetch results for list of malls
results=get_mall_coord(list_of_malls)
mall_location=results[0]
malls_not_found=results[1]
mall_location[['mall_name','road_name']]

100 AM,searchval: 100 AM road_name: TRAS STREET
313@Somerset,searchval: UOB 313@SOMERSET road_name: ORCHARD ROAD
Aperia,searchval: JOSIAH BABIES (APERIA) road_name: KALLANG AVENUE
Balestier Hill Shopping Centre,searchval: BALESTIER HILL SHOPPING CENTRE road_name: THOMSON ROAD
Bugis Cube,searchval: BUGIS CUBE road_name: NORTH BRIDGE ROAD
Bugis Junction,searchval: BUGIS JUNCTION road_name: VICTORIA STREET
Bugis+,searchval: NEW BUGIS STREET road_name: NEW BUGIS STREET
Capitol Piazza,searchval: CAPITOL PIAZZA road_name: STAMFORD ROAD
Cathay Cineleisure Orchard,searchval: UOB CATHAY CINELEISURE ORCHARD road_name: GRANGE ROAD
City Gate,searchval: CITY GATE road_name: BEACH ROAD
City Square Mall,searchval: UOB CITY SQUARE MALL AUTOLOBBY road_name: KITCHENER ROAD
CityLink Mall,searchval: UOB CITYLINK MALL road_name: RAFFLES LINK
The Central,searchval: THE SUITES AT CENTRAL road_name: DEVONSHIRE ROAD
Duo,searchval: DUO RESIDENCES road_name: FRASER STREET
Far East Plaza,searchval: UOB FAR EAST P

No Results
Junction 10,searchval: JUNCTION 10 road_name: WOODLANDS ROAD
Junction 9,searchval: JUNCTION NINE road_name: YISHUN AVENUE 9
Keat Hong Shopping Centre,searchval: KEAT HONG SHOPPING CENTRE road_name: CHOA CHU KANG AVENUE 1
Limbang Shopping Centre,searchval: LIMBANG SHOPPING CENTRE road_name: CHOA CHU KANG STREET 51
Lot One,searchval: SHAW THEATRES LOT ONE road_name: CHOA CHU KANG AVENUE 4
Marsiling Mall,searchval: MARSILING MALL HAWKER CENTRE road_name: WOODLANDS STREET 12
myVillage Serangoon,searchval: MYVILLAGE AT SERANGOON GARDEN road_name: MAJU AVENUE
NEX,searchval: NEX road_name: SERANGOON CENTRAL
Northpoint City,searchval: NORTHPOINT CITY road_name: YISHUN AVENUE 2
Oasis Terraces,searchval: OASIS TERRACES road_name: PUNGGOL DRIVE
Punggol Plaza,searchval: UOB PUNGGOL PLAZA road_name: PUNGGOL FIELD
Rail Mall,searchval: THE RAIL MALL road_name: UPPER BUKIT TIMAH ROAD
Rivervale Mall,searchval: UOB RIVERVALE MALL road_name: RIVERVALE CRESCENT
Rivervale Plaza,searchval: UOB RI

,mall_name,road_name
0,100 AM,TRAS STREET
0,313@Somerset,ORCHARD ROAD
0,Aperia,KALLANG AVENUE
0,Balestier Hill Shopping Centre,THOMSON ROAD
0,Bugis Cube,NORTH BRIDGE ROAD
0,Bugis Junction,VICTORIA STREET
0,Bugis+,NEW BUGIS STREET
0,Capitol Piazza,STAMFORD ROAD
0,Cathay Cineleisure Orchard,GRANGE ROAD
0,City Gate,BEACH ROAD


In [17]:
# Hardcode for malls not found and get coordinates
print(malls_not_found)

['Pomo', 'Jubilee Square']


In [18]:
# Re-fetch results for Pomo and Jubilee Square
malls_round2=['GR.ID','569814'] # Change search values for pomo and jubliee square to these
results_round2=get_mall_coord(malls_round2)
mall_location_round2=results_round2[0]
malls_not_found_round2=results_round2[1]

# See results
malls_not_found_round2
mall_location_round2

GR.ID,searchval: GR.ID road_name: SELEGIE ROAD
569814,searchval: 61 ANG MO KIO AVENUE 8 SINGAPORE 569814 road_name: ANG MO KIO AVENUE 8


,mall_name,searchval,latitude,longitude,postal_code,road_name
0,GR.ID,GR.ID,1.30004685562708,103.849246371869,188306,SELEGIE ROAD
0,569814,61 ANG MO KIO AVENUE 8 SINGAPORE 569814,1.37178551879641,103.847775723803,569814,ANG MO KIO AVENUE 8


In [19]:
# Correct mall name for Jubilee Square
mall_location_round2.loc[mall_location_round2['mall_name']=='569814',['mall_name']]='Jubilee Square'

# Append to main dataframe
mall_location=mall_location.append(mall_location_round2)
mall_location.sort_values(['mall_name'],inplace=True)
mall_location.dropna(inplace=True)

In [20]:
# Save mall locations to CSV
mall_location.to_csv('mall_location.csv',index=False)